# Recipe 1: Directly Comparing Two Reforms

This is an advanced recipe that should be followed only after mastering the basic recipe.
This recipe shows how to compare two reforms (instead of comparing a reform to current-law policy)
and also shows how to use the reform files available on the Tax-Calculator website
(instead of reform files on your computer’s disk).

Print reform2-vs-reform1 difference table